In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#from IPython.display import display
import base64
import string
import re
import nltk

from collections import Counter
from time import time
# from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
url='https://github.com/GU4243-ADS/spring2018-project1-ginnyqg/raw/master/data/spooky.csv'

train = pd.read_csv(url,
                    encoding='latin-1')
train.columns = train.columns.str.capitalize()
                    
train['ID'] = train['Id'].str.replace("id","")
train = train.set_index('Id')

In [3]:
print(train)

                                                      Text Author     ID
Id                                                                      
id26305  This process, however, afforded me no means of...    EAP  26305
id17569  It never once occurred to me that the fumbling...    HPL  17569
id11008  In his left hand was a gold snuff box, from wh...    EAP  11008
id27763  How lovely is spring As we looked from Windsor...    MWS  27763
id12958  Finding nothing else, not even gold, the Super...    HPL  12958
...                                                    ...    ...    ...
id17718  I could have fancied, while I looked at it, th...    EAP  17718
id08973  The lids clenched themselves together as if in...    EAP  08973
id05267  Mais il faut agir that is to say, a Frenchman ...    EAP  05267
id17513  For an item of news like this, it strikes us i...    EAP  17513
id00393  He laid a gnarled claw on my shoulder, and it ...    HPL  00393

[19579 rows x 3 columns]


In [4]:
train.head()

,Text,Author,ID
Id,,,
id26305,"This process, however, afforded me no means of...",EAP,26305
id17569,It never once occurred to me that the fumbling...,HPL,17569
id11008,"In his left hand was a gold snuff box, from wh...",EAP,11008
id27763,How lovely is spring As we looked from Windsor...,MWS,27763
id12958,"Finding nothing else, not even gold, the Super...",HPL,12958


In [5]:
print(train.groupby(["ID","Author"]))

In [6]:
lemma = WordNetLemmatizer()

train_clean = (train
    .groupby(["ID","Author"])
    .apply(lambda s: nltk.word_tokenize(' '.join(s['Text'])))
    .apply(lambda words: [word for word in words if word.isalpha()])
)
print(train_clean)
from nltk.corpus import stopwords  
stop_words = set(stopwords.words('english'))

train_clean = (train_clean
    .apply(lambda words: [lemma.lemmatize(w) for w in words if not w in stop_words])
    .reset_index(name='tokenized')
)

train_clean.head(2)

ID     Author
00001  MWS       [Idris, was, well, content, with, this, resolv...
00002  HPL       [I, was, faint, even, fainter, than, the, hate...
00003  EAP       [Above, all, I, burn, to, know, the, incidents...
00004  EAP       [He, might, see, perhaps, one, or, two, points...
00005  MWS       [All, obeyed, the, Lord, Protector, of, dying,...
                                       ...                        
27965  HPL       [Arter, I, got, the, book, off, Eb, I, uster, ...
27966  EAP       [I, still, lay, quietly, and, made, effort, to...
27967  EAP       [The, ultimate, body, thus, escapes, our, rudi...
27968  MWS       [The, next, morning, Felix, went, out, to, his...
27971  MWS       [I, found, myself, similar, yet, at, the, same...
Length: 19579, dtype: object


,ID,Author,tokenized
0,00001,MWS,"[Idris, well, content, resolve, mine]"
1,00002,HPL,"[I, faint, even, fainter, hateful, modernity, ..."


In [7]:
corpus = train_clean[train_clean["Author"] == "EAP"]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words='english')# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(corpus['tokenized'].apply(lambda s: ' '.join(s)))


number_topics = 11
# Create and fit the LDA model
lda = LatentDirichletAllocation(n_components=number_topics, max_iter=5,
                                learning_method = 'online',
                                learning_offset = 50.,
                                random_state = 0,
                                n_jobs = 1)
lda.fit(count_data)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, n_components=11, n_jobs=1,
                          random_state=0)

In [9]:
print(lda.components_)

[[0.09106879 0.90168427 0.09107049 ... 0.09103489 2.25973382 1.03750852]
 [0.0910431  0.09105079 0.09102889 ... 0.09104547 0.0910583  0.09103278]
 [0.80614144 0.09113431 0.09106276 ... 0.09152101 0.09105682 0.09102169]
 ...
 [0.09104864 0.09102604 0.09113256 ... 0.93598722 0.09104342 0.09475499]
 [0.09103607 0.09117965 0.09105287 ... 0.09102179 0.09102504 0.09102363]
 [0.09132759 0.09107547 0.09104532 ... 0.09153208 0.09105121 0.09103602]]


In [16]:
print(len(lda.components_))
print(len(lda.components_[0]))

11
13396


In [21]:
print(count_data.shape)

(7900, 13396)


In [41]:
lda.__getstate__()

{'n_components': 11,
 'doc_topic_prior': None,
 'topic_word_prior': None,
 'learning_method': 'online',
 'learning_decay': 0.7,
 'learning_offset': 50.0,
 'max_iter': 5,
 'batch_size': 128,
 'evaluate_every': -1,
 'total_samples': 1000000.0,
 'perp_tol': 0.1,
 'mean_change_tol': 0.001,
 'max_doc_update_iter': 100,
 'n_jobs': 1,
 'verbose': 0,
 'random_state': 0,
 'n_features_in_': 13396,
 'random_state_': RandomState(MT19937) at 0x28B40FC1E40,
 'n_batch_iter_': 311,
 'n_iter_': 5,
 'doc_topic_prior_': 0.09090909090909091,
 'topic_word_prior_': 0.09090909090909091,
 'components_': array([[0.09106879, 0.90168427, 0.09107049, ..., 0.09103489, 2.25973382,
         1.03750852],
        [0.0910431 , 0.09105079, 0.09102889, ..., 0.09104547, 0.0910583 ,
         0.09103278],
        [0.80614144, 0.09113431, 0.09106276, ..., 0.09152101, 0.09105682,
         0.09102169],
        ...,
        [0.09104864, 0.09102604, 0.09113256, ..., 0.93598722, 0.09104342,
         0.09475499],
        [0.091036